In [1]:
from typing import List, Optional
from typing_extensions import TypedDict

# 장소 정보를 담는 TypedDict 정의
class KakaoPlace(TypedDict):
    name: str
    address: str
    url: str

# 대화 상태를 담는 TypedDict 정의
class State(TypedDict):
    # 대화 메시지 목록 (Human, AI, Tool 메시지들이 순서대로 저장됨)
    messages: List             # 실제 타입은 아래에서 Annotated를 통해 정의 예정
    # 장소 검색용으로 추출된 키워드 (초기에는 없을 수 있음)
    search_query: Optional[str]
    # Kakao API로 찾은 장소 결과 목록 (초기에는 없을 수 있음)
    search_results: Optional[List[KakaoPlace]]

In [2]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

# State 클래스 수정: messages에 Annotated 적용
from typing import Annotated
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    search_query: Optional[str]
    search_results: Optional[List[KakaoPlace]]

In [3]:
from langchain_openai import ChatOpenAI

# 환경 변수에서 API 키 로드 (OpenAI API와 Kakao API 키)
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수를 불러옴

# OpenAI 챗 모델 초기화
openai_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=openai_model)

In [4]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# 키워드 추출 도구 함수 정의
@tool
def extract_keyword(user_request: str) -> str:
    """사용자 요청에서 장소 검색을 위한 짧고 명확한 키워드를 추출합니다."""
    # 키워드 추출을 위한 프롬프트 작성
    prompt = (
        "다음 요청에서 장소 검색에 사용할 키워드를 한 문장으로 추출해줘.\n"
        "- 키워드는 띄어쓰기로 구분된 한 문장이어야 해.\n"
        "- 다른 설명은 하지 말고 키워드만 알려줘.\n\n"
        f"요청: '{user_request}'"
    )
    # LLM에게 프롬프트 전달하여 응답 받기
    response = llm.invoke([HumanMessage(content=prompt)])
    # 응답에서 키워드 추출 (따옴표나 공백 제거)
    keyword = response.content.strip().strip('"').strip("'")
    return keyword

In [ ]:
import requests  # HTTP 요청을 위해 필요

@tool
def kakao_place_search(query: str) -> list[KakaoPlace]:
    """
    카카오 장소 검색 API를 사용하여 장소를 검색합니다.
    
    Args:
        query: 검색할 장소 키워드 (예: '강남역 맛집', '홍대 카페', '서울 박물관' 등)
              구체적인 지역명과 장소 유형을 함께 입력하면 더 정확한 결과를 얻을 수 있습니다.
    
    Returns:
        List[KakaoPlace]: 검색된 장소 정보 목록 (최대 5개)
        각 장소 정보는 다음 필드를 포함합니다:
        - name: 장소명
        - address: 장소 주소
        - url: 장소 상세 정보 URL
    
    Example:
        kakao_place_search("서울역 근처 식당")
    """
    # Kakao API 요청을 위한 URL과 헤더, 파라미터 설정
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {"query": query, "size": 5}
    # API 호출
    response = requests.get(url, headers=headers, params=params)
    data = response.json().get("documents", [])
    # 필요한 정보만 추출하여 KakaoPlace 리스트 구성
    results: list[KakaoPlace] = []
    for place in data:
        results.append({
            "name": place.get("place_name", ""),
            "address": place.get("address_name", ""),
            "url": place.get("place_url", "")
        })
    return results

In [6]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 6-1: 각 노드의 동작 정의
def chatbot_node(state: State):
    """LLM이 현재 메시지들을 보고 다음 응답(또는 도구 요청)을 생성하는 노드"""
    # state["messages"]에는 대화의 모든 메시지가 있음 (마지막이 사용자의 최신 질문)
    ai_response = llm_with_tools.invoke(state["messages"])
    # 새로 생성된 AI의 응답 메시지만 돌려주면, Annotated add_messages로 자동 추가됨
    return {"messages": [ai_response]}

# ToolNode 생성 (우리가 정의한 tools 리스트 사용)
tools = [kakao_place_search]    # 도구 리스트 준비
llm_with_tools = llm.bind_tools(tools)           # LLM에 도구 결합
tool_node = ToolNode(tools)                      # ToolNode 인스턴스 생성

# 6-2: 그래프 초기화 및 노드 추가
graph_builder = StateGraph(State)                # 우리 State 타입에 맞는 그래프 생성
graph_builder.add_node("chatbot", chatbot_node)  # "chatbot" 노드 추가
graph_builder.add_node("tools", tool_node)       # "tools" 노드 추가


# 6-3: 노드 사이의 엣지(전이) 정의
# 시작 지점 지정 및 그래프 컴파일
graph_builder.set_entry_point("chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile()

In [7]:
from langchain_core.messages import HumanMessage

# 초기 상태 설정: 사용자 질문 입력
initial_state: State = {
    "messages": [HumanMessage(content="대전을 대표하는 빵집은?")],
    "search_query": None,
    "search_results": None
}

# 그래프 실행하여 결과 얻기
final_state = graph.invoke(initial_state)

In [8]:
from langchain_core.messages import AIMessage, ToolMessage

for msg in final_state["messages"]:
    if isinstance(msg, HumanMessage):
        print("👤 사용자:", msg.content)
    elif isinstance(msg, AIMessage):
        print("🤖 챗봇:", msg.content)
        if msg.tool_calls:
            # AI 메세지에 도구 호출 정보가 있을 경우
            for call in msg.tool_calls:
                print(f"  ↪ (도구 요청: {call['name']} {call['args']})")
    elif isinstance(msg, ToolMessage):
        # 도구 실행 결과 메시지
        print(f"🔧 도구[{msg.name}] 결과:", msg.content)

👤 사용자: 대전을 대표하는 빵집은?
🤖 챗봇: 
  ↪ (도구 요청: kakao_place_search {'query': '대전 빵집'})
🔧 도구[kakao_place_search] 결과: [{"name": "성심당 본점", "address": "대전 중구 은행동 145-1", "url": "http://place.map.kakao.com/17733090"}, {"name": "성심당 대전역점", "address": "대전 동구 정동 1-1", "url": "http://place.map.kakao.com/21537026"}, {"name": "달달빵집", "address": "대전 중구 목동 111-12", "url": "http://place.map.kakao.com/1063471907"}, {"name": "성심당 롯데백화점 대전점", "address": "대전 서구 괴정동 423-1", "url": "http://place.map.kakao.com/1513470800"}, {"name": "내가만든미케익 대전점", "address": "대전 중구 은행동 52-10", "url": "http://place.map.kakao.com/10184597"}]
🤖 챗봇: 대전에서 유명한 빵집은 다음과 같습니다:

1. **성심당 본점**
   - 주소: 대전 중구 은행동 145-1
   - [지도 보기](http://place.map.kakao.com/17733090)

2. **성심당 대전역점**
   - 주소: 대전 동구 정동 1-1
   - [지도 보기](http://place.map.kakao.com/21537026)

3. **달달빵집**
   - 주소: 대전 중구 목동 111-12
   - [지도 보기](http://place.map.kakao.com/1063471907)

4. **성심당 롯데백화점 대전점**
   - 주소: 대전 서구 괴정동 423-1
   - [지도 보기](http://place.map.kakao.com/1513470800)



In [9]:
initial_state["messages"] = [HumanMessage(content="서울에서 애견동반 가능한 카페 알려줘")]
final_state = graph.invoke(initial_state)
# 위와 같은 방법으로 final_state["messages"]를 출력하여 확인

In [10]:
for msg in final_state["messages"]:
    if isinstance(msg, HumanMessage):
        print("👤 사용자:", msg.content)
    elif isinstance(msg, AIMessage):
        print("🤖 챗봇:", msg.content)
        if msg.tool_calls:
            # AI 메세지에 도구 호출 정보가 있을 경우
            for call in msg.tool_calls:
                print(f"  ↪ (도구 요청: {call['name']} {call['args']})")
    elif isinstance(msg, ToolMessage):
        # 도구 실행 결과 메시지
        print(f"🔧 도구[{msg.name}] 결과:", msg.content)

👤 사용자: 서울에서 애견동반 가능한 카페 알려줘
🤖 챗봇: 
  ↪ (도구 요청: kakao_place_search {'query': '서울 애견동반 카페'})
🔧 도구[kakao_place_search] 결과: [{"name": "프릳츠 도화점", "address": "서울 마포구 도화동 179-9", "url": "http://place.map.kakao.com/24529429"}, {"name": "할아버지공장", "address": "서울 성동구 성수동2가 309-133", "url": "http://place.map.kakao.com/1301973155"}, {"name": "꽁티드툴레아 도산점", "address": "서울 강남구 신사동 646-21", "url": "http://place.map.kakao.com/179244712"}, {"name": "마일스톤커피", "address": "서울 강남구 신사동 554-4", "url": "http://place.map.kakao.com/22549791"}, {"name": "로우키", "address": "서울 성동구 성수동2가 302-16", "url": "http://place.map.kakao.com/1302549042"}]
🤖 챗봇: 서울에서 애견동반 가능한 카페 몇 곳을 소개합니다:

1. **프릳츠 도화점**
   - 주소: 서울 마포구 도화동 179-9
   - [지도 링크](http://place.map.kakao.com/24529429)

2. **할아버지공장**
   - 주소: 서울 성동구 성수동2가 309-133
   - [지도 링크](http://place.map.kakao.com/1301973155)

3. **꽁티드툴레아 도산점**
   - 주소: 서울 강남구 신사동 646-21
   - [지도 링크](http://place.map.kakao.com/179244712)

4. **마일스톤커피**
   - 주소: 서울 강남구 신사동 554-4
   - [지도 링크](http